In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from Tight_Binding.tight_binding.extract_parameter import input_data
from Tight_Binding.tight_binding.extract_parameter import extract_parameter
from Tight_Binding.tight_binding.hamiltonian import hamiltonian
from Tight_Binding.tight_binding.hamiltonian import multiple_hamiltonian

# Import Data

In [2]:
loc_parameter = "Data/SrNbO3/hr_files/srnbo3_hr_4x4x4.dat"
loc_atom_position = "Data/SrNbO3/atomic_position.csv"
loc_orbital_index = "Data/SrNbO3/orbital_index.csv"

In [3]:
parameter = input_data(loc_parameter, loc_atom_position, loc_orbital_index, 
           a = 4.089,
           b = 4.089,
           c = 4.089)

In [4]:
parameter.get_parameter_TB().head()

,X,Y,Z,A,B,Re,Im
0,-2,-2,-2,1,1,-0.002093,-0.000000
1,-2,-2,-2,2,1,-0.003924,-0.002397
2,-2,-2,-2,3,1,0.001737,0.001976
3,-2,-2,-2,4,1,-0.002321,-0.000058
4,-2,-2,-2,5,1,0.001143,-0.004803


In [5]:
input_hamiltonian = parameter.vec_lattice()
input_hamiltonian.head()

,X,Y,Z,A,B,Re,Im,Rx,Ry,Rz
0,-2,-2,-2,1,1,-0.002093,-0.000000,-8.178,-8.178,-8.178
1,-2,-2,-1,1,1,-0.001895,0.001362,-8.178,-8.178,-4.089
2,-2,-2,0,1,1,-0.003405,0.000000,-8.178,-8.178,0.000
3,-2,-2,1,1,1,-0.001895,-0.001362,-8.178,-8.178,4.089
4,-2,-2,2,1,1,-0.002093,-0.000000,-8.178,-8.178,8.178


In [104]:
matriks = input_hamiltonian.to_numpy()
matriks

array([[-2.   , -2.   , -2.   , ..., -8.178, -8.178, -8.178],
       [-2.   , -2.   , -1.   , ..., -8.178, -8.178, -4.089],
       [-2.   , -2.   ,  0.   , ..., -8.178, -8.178,  0.   ],
       ...,
       [ 2.   ,  2.   ,  0.   , ...,  8.178,  8.178,  0.   ],
       [ 2.   ,  2.   ,  1.   , ...,  8.178,  8.178,  4.089],
       [ 2.   ,  2.   ,  2.   , ...,  8.178,  8.178,  8.178]])

In [108]:
k = np.array([2.0,2.0,2.0])

hamiltonian = np.zeros(shape=(len(a.A.unique()), len(a.B.unique())),
                        dtype=np.complex128)

In [123]:
start = time.time()
for i, A in enumerate(np.unique(matriks[:,3])):
    for j, B in enumerate(np.unique(matriks[:,4])):
        df = matriks[(matriks[:,3]==A) & (matriks[:,4]==B)]
        
        R = np.array([df[:,7], df[:,8], df[:,9]])
        sum_H = (df[:,5] + df[:,6]*1j)*np.exp(-1j*np.dot(k,R))
        sum_H = sum_H.sum()
        
        hamiltonian[i,j] = sum_H
print(time.time() - start, "Detik")

print(hamiltonian[3,13], hamiltonian[13,3])

0.47365903854370117 Detik
(-0.1554659234998727+0.02814627140589869j) (-0.1554659234998727-0.028146271405898694j)


In [124]:
a = input_hamiltonian

start = time.time()
for i, A in enumerate(a.A.unique()):
    for j, B in enumerate(a.B.unique()):
        filter_A = a.A == A
        filter_B = a.B == B
        df = a[filter_A & filter_B]

        R = np.array([df.Rx, df.Ry, df.Rz])
        sum_H = (df.Re + df.Im*1j)*np.exp(-1j*np.dot(k,R))
        sum_H = sum_H.sum()

        hamiltonian[i,j] = sum_H
print(time.time() - start, "Detik")

print(hamiltonian[3,13], hamiltonian[13,3])

2.93554425239563 Detik
(-0.1554659234998727+0.02814627140589869j) (-0.1554659234998727-0.028146271405898694j)


# Future Improvements

# Multiple kpoints

In [6]:
k_grid = np.ones(shape=(8,3))

In [9]:
start = time.time()
ham_1 = multiple_hamiltonian(k_grid, input_hamiltonian, hamiltonian)
print(time.time() - start, "Detik")

print(ham_1[3,3,13], ham_1[3,13,3])

5.4796247482299805 Detik
(0.01935444999240512+0.11637411719308206j) (0.019354449992405105-0.11637411719308209j)


In [13]:
start = time.time()
ham_2 = multiple_hamiltonian(k_grid, input_hamiltonian, hamiltonian)
print(time.time() - start, "Detik")

print(ham_2[3,3,13], ham_2[3,13,3])

1.4097955226898193 Detik
(0.01935444999240512+0.11637411719308206j) (0.019354449992405105-0.11637411719308209j)
